In [1]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

In [35]:
# Define a tool
async def get_weather(city: str) -> str:
    return f"The weather in {city} is 73 degrees and Sunny."

In [36]:
async def main() -> None:
    # Define an agent
    weather_agent = AssistantAgent(
        name="weather_agent",
        model_client=OpenAIChatCompletionClient(
            model="ollama/llama3.2:3b",
            api_key="YOUR_API_KEY",
            base_url="http://0.0.0.0:4000/",
            model_capabilities={
            "json_output": False,
            "vision": False,
            "function_calling": True,
        },
        ),
        tools=[get_weather],
    )

    # Define a team with a single agent and maximum auto-gen turns of 1.
    agent_team = RoundRobinGroupChat([weather_agent], max_turns=1)

    while True:
        # Get user input from the console.
        user_input = input("Enter a message (type 'exit' to leave): ")
        if user_input.strip().lower() == "exit":
            break
        # Run the team and stream messages to the console.
        stream = agent_team.run_stream(task=user_input)
        await Console(stream)

In [37]:
await main()

---------- user ----------
SF
---------- weather_agent ----------
[FunctionCall(id='call_8f268ee1-deb6-4329-b60e-39307565f728', arguments='{"city": "SF"}', name='get_weather')]
[Prompt tokens: 154, Completion tokens: 16]
---------- weather_agent ----------
[FunctionExecutionResult(content='The weather in SF is 73 degrees and Sunny.', call_id='call_8f268ee1-deb6-4329-b60e-39307565f728')]
---------- weather_agent ----------
The weather in SF is 73 degrees and Sunny.
---------- Summary ----------
Number of messages: 4
Finish reason: Maximum number of turns 1 reached.
Total prompt tokens: 154
Total completion tokens: 16
Duration: 0.52 seconds
